In [1]:
import os 
import wget
import gzip
import random
import scipy
import tensorflow as tf
from collections import defaultdict
from implicit import bpr
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split

/home/zhankui/anaconda3/envs/pml/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data is available at http://cseweb.ucsd.edu/~jmcauley/pml/data/. 
- Download and save to your own directory.
- Or, run following script to save it into `Chapter_5/data` folder automatically.

In [2]:
filenames = [
    'goodreads_fantasy.tsv', 
    'goodreads_reviews_fantasy_paranormal.json.gz', 
    'goodreads_reviews_comics_graphic.json.gz'
]

dataDir = './data'
url = 'http://jmcauley.ucsd.edu/pml_data'

if not os.path.exists(dataDir):
    os.makedirs(dataDir)
for filename in filenames:
    wget.download(os.path.join(url, filename), out=dataDir)
print("Done!")

Done!


# Latent factor model (Surprise)

Using the library's inbuilt data reader, extract tsv-formatted data

In [3]:
reader = Reader(line_format='user item rating', sep='\t')
data = Dataset.load_from_file(os.path.join(dataDir, "goodreads_fantasy.tsv"), reader=reader)

Standard latent-factor model

In [4]:
model = SVD()

Inbuilt functions to split into training and test fractions

In [5]:
trainset, testset = train_test_split(data, test_size=.25)

Fit the model and extract predictions

In [6]:
model.fit(trainset)
predictions = model.test(testset)

Estimate for a single (test) rating

In [8]:
predictions[0].est

3.7170605528633813

MSE for model predictions (test set)

In [9]:
sse = 0
for p in predictions:
    sse += (p.r_ui - p.est)**2

print(sse / len(predictions))

1.1844525879282073


# Bayesian Personalized Ranking (Implicit)

In [10]:
def parseData(fname):
    for l in gzip.open(fname):
        d = eval(l)
        del d['review_text'] # Discard the reviews, to save memory when we don't use them
        yield d

Full dataset of Goodreads fantasy reviews (fairly memory-hungry, could be replaced by something smaller)

In [11]:
data = list(parseData(os.path.join(dataDir, "goodreads_reviews_fantasy_paranormal.json.gz")))

In [13]:
random.shuffle(data)

Example from the dataset

In [14]:
data[0]

{'user_id': '9f4e4bd584d5833110e994881ac483d3',
 'book_id': '2260675',
 'review_id': '49873a9cd35b0553df41a1242797a21f',
 'rating': 5,
 'date_added': 'Thu Aug 21 00:36:13 -0700 2008',
 'date_updated': 'Sat Feb 07 17:53:22 -0800 2009',
 'read_at': 'Wed Aug 20 00:00:00 -0700 2008',
 'started_at': '',
 'n_votes': 0,
 'n_comments': 0}

Build a few utility data structures. Since we'll be converting the data to a sparse interaction matrix, the main structure here is to assign each user/item to an ID from 0 to nUsers/nItems.

In [15]:
userIDs,itemIDs = {},{}

for d in data:
    u,i = d['user_id'],d['book_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

In [16]:
nUsers,nItems

(256088, 258212)

Convert dataset to sparse matrix. Only storing positive feedback instances (i.e., rated items).

In [17]:
Xiu = scipy.sparse.lil_matrix((nItems, nUsers))
for d in data:
    Xiu[itemIDs[d['book_id']],userIDs[d['user_id']]] = 1
    
Xui = scipy.sparse.csr_matrix(Xiu.T)

Bayesian Personalized Ranking model with 5 latent factors

In [18]:
model = bpr.BayesianPersonalizedRanking(factors = 5)

Fit the model

In [19]:
model.fit(Xiu)

AttributeError: has_sorted_indices not found

Get recommendations for a particular user (the first one) and to get items related to (similar latent factors) to a particular item

In [18]:
recommended = model.recommend(0, Xui)
related = model.similar_items(0)

In [19]:
related

[(0, 1.0),
 (42098, 0.9885355),
 (142964, 0.9845209),
 (150861, 0.98274595),
 (231639, 0.9826295),
 (182330, 0.9813926),
 (240868, 0.98134804),
 (226720, 0.9796706),
 (84748, 0.9783791),
 (140340, 0.97788805)]

Extract user and item factors

In [20]:
itemFactors = model.item_factors
userFactors = model.user_factors

In [21]:
itemFactors[0]

array([-0.74582803, -0.10878776,  0.32922822,  0.16516064,  0.38874012,
        0.7460656 ], dtype=float32)

# Latent factor model (Tensorflow)

In [20]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

Goodreads comic book data

In [22]:
userIDs = {}
itemIDs = {}
interactions = []

for d in parse(os.path.join(dataDir, "goodreads_reviews_comics_graphic.json.gz")):
    u = d['user_id']
    i = d['book_id']
    r = d['rating']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))

In [23]:
random.shuffle(interactions)
len(interactions)

542338

Split into train and test sets

In [24]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [25]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

Mean rating, just for initialization

In [26]:
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

Gradient descent optimizer, could experiment with learning rate

In [27]:
optimizer = tf.keras.optimizers.Adam(0.1)

Latent factor model tensorflow class

In [28]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

Initialize the model. Could experiment with number of factors and regularization rate.

In [30]:
modelLFM = LatentFactorModel(mu, 5, 0.00001)

Training step (for the batch-based model from Chapter 5)

In [31]:
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

Run 100 iterations (really 100 batches) of gradient descent

In [32]:
for i in range(100):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.52663964
iteration 20, objective = 0.50974727
iteration 30, objective = 0.5178934
iteration 40, objective = 0.51433665
iteration 50, objective = 0.5182813
iteration 60, objective = 0.51444495
iteration 70, objective = 0.5095492
iteration 80, objective = 0.5071063
iteration 90, objective = 0.5167384
iteration 100, objective = 0.5054537


Prediction for a particular user/item pair

In [33]:
u,i,r = interactionsTest[0]

In [34]:
modelLFM.predict(userIDs[u], itemIDs[i]).numpy()

3.5586188

# Bayesian personalized ranking (Tensorflow)

In [35]:
items = list(itemIDs.keys())

Batch-based version from Chapter 5

In [36]:
class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

In [37]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [38]:
modelBPR = BPRbatch(5, 0.00001)

In [39]:
def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in itemsPerUser[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

Run 100 batches of gradient descent

In [40]:
for i in range(100):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.5298456
iteration 20, objective = 0.47559565
iteration 30, objective = 0.47294614
iteration 40, objective = 0.4711959
iteration 50, objective = 0.47494137
iteration 60, objective = 0.4759138
iteration 70, objective = 0.47475028
iteration 80, objective = 0.4721527
iteration 90, objective = 0.47154135
iteration 100, objective = 0.4697961


Prediction for a particular user/item pair. Note that this is an unnormalized score (which can be used for ranking)

In [41]:
u,i,_ = interactionsTest[0]

In [42]:
# In this case just a score (that can be used for ranking), rather than a prediction of a rating
modelBPR.predict(userIDs[u], itemIDs[i]).numpy()

0.40530765

# Exercises

### 5.1

Adapt the latent factor model above, simply deleting any terms associated with latent factors

In [43]:
class LatentFactorModelBiasOnly(tf.keras.Model):
    def __init__(self, mu, lamb):
        super(LatentFactorModelBiasOnly, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i]
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        pred = self.alpha + beta_u + beta_i
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [44]:
modelBiasOnly = LatentFactorModelBiasOnly(mu, 0.00001)

In [45]:
def trainingStepBiasOnly(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
        (grad, var) in zip(gradients, model.trainable_variables)
        if grad is not None)
    return loss.numpy()

In [46]:
for i in range(50):
    obj = trainingStepBiasOnly(modelBiasOnly, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.57272446
iteration 20, objective = 0.54454523
iteration 30, objective = 0.53504544
iteration 40, objective = 0.52635914
iteration 50, objective = 0.5215947


Compute the MSEs for a model which always predicts the mean, versus one which involves bias terms

In [47]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [48]:
alwaysPredictMean = [mu for _ in interactionsTest]
labels = [r for _,_,r in interactionsTest]

In [49]:
MSE(alwaysPredictMean, labels)

1.3187948295868135

In [50]:
biasOnlyPredictions =\
    [modelBiasOnly.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in interactionsTest]

In [51]:
biasOnlyPredictions[0]

3.4326916

In [52]:
MSE(biasOnlyPredictions, labels)

0.9908003582674878

### 5.2

Performance of a complete latent factor model (using the latent factor model implementation in the examples above)

In [53]:
optimizer = tf.keras.optimizers.Adam(0.1)
modelLFM = LatentFactorModel(mu, 10, 0.00001)

In [54]:
for i in range(50):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.5290122
iteration 20, objective = 0.53150094
iteration 30, objective = 0.5345895
iteration 40, objective = 0.5308519
iteration 50, objective = 0.53291863


In [55]:
predictions = [modelLFM.predict(userIDs[u],itemIDs[i]).numpy() for u,i,_ in interactionsTest]

In [56]:
MSE(predictions, labels)

1.0002061847676786

(probably needs a little more tuning in terms of number of latent factors, learning rate, etc.)

### 5.3

Experiment with rounding the predictions

In [57]:
predictionsRounded = [int(p + 0.5) for p in predictions]

In [58]:
MSE(predictionsRounded, labels)

1.0830106575211123

Seems to result in worse performance. For a rough explanation, consider a random variable that takes a value of "1" half the time and "2" half the time; in terms of the MSE, always predicting 1.5 (and always incurring moderate errors) is preferable to always predicting either of 1 or 2 (and incurring a large error half the time).

### 5.4

Following the BPR code from examples above

In [59]:
optimizer = tf.keras.optimizers.Adam(0.1)
modelBPR = BPRbatch(10, 0.00001)

In [60]:
for i in range(50):
    obj = trainingStepBPR(modelBPR, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.5278025
iteration 20, objective = 0.48860368
iteration 30, objective = 0.4848184
iteration 40, objective = 0.48684
iteration 50, objective = 0.49009293


In [61]:
interactionsTestPerUser = defaultdict(set)
itemSet = set()
for u,i,_ in interactionsTest:
    interactionsTestPerUser[u].add(i)
    itemSet.add(i)

AUC implementation

In [62]:
def AUCu(u, N): # N samples per user
    win = 0
    if N > len(interactionsTestPerUser[u]):
        N = len(interactionsTestPerUser[u])
    positive = random.sample(interactionsTestPerUser[u],N)
    negative = random.sample(itemSet.difference(interactionsTestPerUser[u]),N)
    for i,j in zip(positive,negative):
        si = modelBPR.predict(userIDs[u], itemIDs[i]).numpy()
        sj = modelBPR.predict(userIDs[u], itemIDs[j]).numpy()
        if si > sj:
            win += 1
    return win/N

In [63]:
def AUC():
    av = []
    for u in interactionsTestPerUser:
        av.append(AUCu(u, 10))
    return sum(av) / len(av)

In [64]:
AUC()

0.7884310302103841